In [1]:
# calculate transition matrix for each section

import datetime
import pandas as pd

In [2]:
# correct data (customers with no marked checkout)

def missing_checkout(data): # fixes data quality issue:
                            # last customers of the day are missing from checkout

    data["timestamp"] = pd.to_datetime(data["timestamp"])
    data.set_index("timestamp", inplace=True)

    checkout = set(data[data["location"] == "checkout"]
                   ["customer_no"].unique())
    all_c = set(data["customer_no"].unique())
    diff = all_c.difference(checkout)

    current_date = data.index.date[0]
    new_time_stemp = f"{current_date} 22:00:00"

    data.reset_index(inplace=True)

    for cust in diff:
        data = data.append({"timestamp": pd.to_datetime(new_time_stemp),
                            "customer_no": cust,
                            "location": "checkout"},
                            ignore_index=True)
    return data

In [3]:
# each customer starts from initial state entrance / insert this state in the data

def inserting_initial_state(data):
    min_datetime = data.groupby("customer_no")["timestamp"].first().reset_index()
    one_minute = datetime.timedelta(minutes=1)

    for i in range(min_datetime.shape[0]):
        data = data.append({"timestamp": min_datetime["timestamp"].iloc[i] - one_minute, 
                            "customer_no": min_datetime["customer_no"].iloc[i],
                            "location": "entrance"},
                            ignore_index=True)
    return data

In [4]:
# make customer name unique

def cust_no_name(data, weekday):
    data["customer_no"] = data["customer_no"].apply(lambda x: str(x)+"_"+weekday)
    return data

In [5]:
# read in data for each weekdays

monday = pd.read_csv("./data/monday.csv",
                     header=0, sep=";", parse_dates=True)
monday = missing_checkout(monday)
monday["weekday"] = "monday"
cust_no_name(monday, "monday")

tuesday = pd.read_csv("./data/tuesday.csv",
                      header=0, sep=";", parse_dates=True)
tuesday = missing_checkout(tuesday)
tuesday["weekday"] = "tuesday"
cust_no_name(tuesday, "tuesday")

wednesday = pd.read_csv("./data/wednesday.csv",
                        header=0, sep=";", parse_dates=True)
wednesday = missing_checkout(wednesday)
wednesday["weekday"] = "wednesday"
cust_no_name(wednesday, "wednesday")

thursday = pd.read_csv("./data/thursday.csv",
                       header=0, sep=";", parse_dates=True)
thursday = missing_checkout(thursday)
thursday["weekday"] = "thursday"
cust_no_name(thursday, "thursday")

friday = pd.read_csv("./data/thursday.csv", header=0,
                     sep=";", parse_dates=True)
friday = missing_checkout(friday)
friday["weekday"] = "friday"
cust_no_name(friday, "friday")

,timestamp,customer_no,location,weekday
0,2019-09-05 07:00:00,1_friday,drinks,friday
1,2019-09-05 07:01:00,2_friday,fruit,friday
2,2019-09-05 07:01:00,3_friday,dairy,friday
3,2019-09-05 07:01:00,4_friday,dairy,friday
4,2019-09-05 07:02:00,1_friday,checkout,friday
...,...,...,...,...
5102,2019-09-05 21:49:00,1534_friday,checkout,friday
5103,2019-09-05 21:49:00,1535_friday,checkout,friday
5104,2019-09-05 22:00:00,1532_friday,checkout,friday
5105,2019-09-05 22:00:00,1533_friday,checkout,friday


In [6]:
# concatenate each weekday in one data frame

data = pd.concat([monday, tuesday, wednesday, thursday, friday])

In [7]:
# inserting initial state

data = inserting_initial_state(data)
data.set_index("timestamp", inplace=True)
data = data.groupby("customer_no").resample(rule="1T").ffill()

In [8]:
# reset index to sort correctly

del data["customer_no"]
data.reset_index(inplace=True)

In [9]:
# values need to be sorted inplace, such that we have
# correct transitions
# calculate next location (transition from one section to the other)

data.sort_values(["customer_no", "timestamp"], inplace=True)
data["location_next"] = data["location"].shift(-1)

In [10]:
# checkout will only migrate to checkout!

data.loc[(data.location == 'checkout'), 'location_next'] = 'checkout'

In [11]:
# collect the final states

states = data[["location", "location_next"]]

In [12]:
# calculate transition matrix

transition_matrix = pd.crosstab(data['location'],
                                data['location_next'],
                                normalize=0)

In [13]:
# check if the matrix is correct

transition_matrix.sum(axis=1)

location
checkout    1.0
dairy       1.0
drinks      1.0
entrance    1.0
fruit       1.0
spices      1.0
dtype: float64

In [15]:
# export the transition prob matrix to csv

transition_matrix.to_csv("./data/transition_matrix.csv")
print(transition_matrix)

location_next  checkout     dairy    drinks     fruit    spices
location                                                       
checkout       1.000000  0.000000  0.000000  0.000000  0.000000
dairy          0.102043  0.740514  0.056666  0.050058  0.050719
drinks         0.217556  0.011395  0.595168  0.088943  0.086938
entrance       0.000000  0.293976  0.150870  0.373494  0.181660
fruit          0.204200  0.092540  0.052265  0.599906  0.051089
spices         0.149425  0.193198  0.159660  0.088805  0.408912
